In [1]:
import numpy as np
import pandas as pd

from sklearn.impute import (SimpleImputer,KNNImputer)
from sklearn.ensemble import (RandomForestRegressor, IsolationForest)
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import ExtraTreesRegressor
import lightgbm as lgb

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, DotProduct, WhiteKernel, RationalQuadratic, Matern
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def fill_missing_values(X, n_neighbors = 75, method="KNN"): 
    
    # normalization
    X_std = np.nanstd(X,axis=0,keepdims=True)
    X_ave = np.nanmean(X,axis=0,keepdims=True)
    X_norma = (X-X_ave)/X_std
    
    # use KNNImputer
    imputer = KNNImputer(missing_values=np.nan, n_neighbors=n_neighbors, weights = 'distance') if method=="KNN"\
        else SimpleImputer(missing_values=np.nan, strategy='median')
    
    X_norma_fixed = imputer.fit_transform(X_norma)
    
    return X_norma_fixed

def remove_outliers(X, y):
    print("IsolationForest-Traing data shape before removed: {}".format(X.shape))
    Z = np.c_[X, y]
    iforest = IsolationForest(max_samples=200, random_state=1, contamination=0.005)
    iforest.fit(Z)
    iforest_outlier_pred = iforest.predict(Z)
    
    Z = np.c_[X, y]
    local = LocalOutlierFactor(n_neighbors=150, contamination=0.005)
    local.fit(Z)
    local_outlier_pred = local.fit_predict(Z)

    mask = np.logical_and((iforest_outlier_pred!=-1), (local_outlier_pred!=-1))
    X , y = X[mask, :], y[mask]
    print("IsolationForest-Traing data shape after removed: {}".format(X.shape))
    return X, y

def select_features(X, y, X_test, feature_num=50, n_estimators=80):
    rf = RandomForestRegressor(n_jobs=-1, n_estimators=80, random_state=1)
    rf.fit(X, y)
    indices = np.asarray(list(rf.feature_importances_)).argsort()[-feature_num:][::-1]
    
    X = np.take(X, indices, axis = 1)
    X_test = np.take(X_test, indices, axis = 1)
    return X, X_test

In [3]:

X_train_data = pd.read_csv('X_train.csv')
y_train_data = pd.read_csv('y_train.csv')
X_test_data = pd.read_csv('X_test.csv')

indices_test = np.array(X_test_data)[:,0]
X_test = np.array(X_test_data)[:,1:]
y_train = np.array(y_train_data)[:,1]
X_train = np.array(X_train_data)[:,1:]

## 1. Imputation of Missing Values
* [Reference](https://scikit-learn.org/stable/modules/impute.html)
* We use median of column instead of mean

In [4]:
X_train_missing_indices = X_train[X_train==np]
X_train = fill_missing_values(X_train, n_neighbors=75)
X_test_processed = fill_missing_values(X_test)
print(X_train.shape)
print(X_test_processed.shape)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  


(1212, 828)
(776, 828)


## 2. Outlier Detection
* [reference_sklearn](https://scikit-learn.org/stable/modules/outlier_detection.html)
* [reference_in_detail](https://practicaldatascience.co.uk/machine-learning/how-to-use-the-isolation-forest-model-for-outlier-detection)

In [5]:
X_processed,y_processed = remove_outliers(X_train,y_train)

IsolationForest-Traing data shape before removed: (1212, 828)
IsolationForest-Traing data shape after removed: (1200, 828)


## 3. Feature Selection

In [6]:
# pearsonr feature selection
from scipy.stats import pearsonr
from sklearn.decomposition import PCA

def pearsonr_feature_selection(X, y, X_test,degree,weights):
    feature_idx =  np.array([i for i in range(0,X.shape[1])])
    correlations = {}
#     print("feature_idx: ", feature_idx.T)
    for idx in feature_idx:
        correlation = 0
        for degree_idx in range(0,len(weights)):
            correlation += abs(weights[degree_idx]*pearsonr(X[:,idx],y**degree[degree_idx])[0])
        correlations[idx] = correlation
    data_correlations = pd.DataFrame(correlations, index=['correlation']).T
    indices_desc = data_correlations['correlation'].abs().sort_values(ascending=False).index
#     print(indices_desc)
    return data_correlations,indices_desc

def pearsonr_feature_selection_correlation(X, X_test,threshold):
    feature_idx =  np.array([i for i in range(0,X.shape[1])])
    correlation_select = np.array([True for i in range(0,X.shape[1])])
    for f_idx in feature_idx:
        if(correlation_select[f_idx]):
            feature_co,indices_desc = pearsonr_feature_selection(X[:,(f_idx+1):], X[:,f_idx], X_test_processed,np.array([1]),np.array([1]))
            reomove_features = feature_co.loc[indices_desc][abs(feature_co['correlation']) >= threshold]
            remove_index = reomove_features.index+(f_idx+1)
            correlation_select[remove_index] = False
    
    return X[:,correlation_select],X_test[:,correlation_select]

In [7]:
def LGB_feature_selection(X_processed,X_test_processed,y_processed):
    data_relation,indices_desc = pearsonr_feature_selection(X_processed, y_processed, X_test_processed,np.array([1,-3,3]),np.array([0.9,0.05,0.05]))
    selected_features = data_relation.loc[indices_desc][abs(data_relation['correlation']) >= 0.025]
    X_selection = X_processed[:,selected_features.index]
    X_test_selection = X_test_processed[:,selected_features.index]
    X_selection_tree, X_test_selection_tree = select_features(X_selection, y_processed, X_test_selection,feature_num = 105)
    X_selection_tree_cor, X_test_selection_tree_cor = pearsonr_feature_selection_correlation(X_selection_tree, X_test_selection_tree,0.93)
    return X_selection_tree_cor,X_test_selection_tree_cor

In [8]:
def GP_feature_selection(X_processed,X_test_processed,y_processed, tree_num = 43):
    data_relation,indices_desc = pearsonr_feature_selection(X_processed, y_processed, X_test_processed,np.array([1,-3,3]),np.array([0.95,0.025,0.025]))
    selected_features = data_relation.loc[indices_desc][abs(data_relation['correlation']) >= 0.001]
    X_selection = X_processed[:,selected_features.index]
    X_test_selection = X_test_processed[:,selected_features.index]
    X_selection_tree, X_test_selection_tree = select_features(X_selection, y_processed, X_test_selection,feature_num = tree_num)
    X_selection_tree_cor, X_test_selection_tree_cor = pearsonr_feature_selection_correlation(X_selection_tree, X_test_selection_tree,0.96)
    return X_selection_tree_cor,X_test_selection_tree_cor

In [9]:
def GP_rbf_feature_selection(X_processed,X_test_processed,y_processed):
    data_relation,indices_desc = pearsonr_feature_selection(X_processed, y_processed, X_test_processed,np.array([1,-3,3]),np.array([0.95,0.025,0.025]))
    selected_features = data_relation.loc[indices_desc][abs(data_relation['correlation']) >= 0.001]
    X_selection = X_processed[:,selected_features.index]
    X_test_selection = X_test_processed[:,selected_features.index]
    X_selection_tree, X_test_selection_tree = select_features(X_selection, y_processed, X_test_selection,feature_num = 54)
    X_selection_tree_cor, X_test_selection_tree_cor = pearsonr_feature_selection_correlation(X_selection_tree, X_test_selection_tree,0.96)
    return X_selection_tree_cor,X_test_selection_tree_cor

In [10]:
def Tree_feature_selection(X_processed,X_test_processed,y_processed):
    data_relation,indices_desc = pearsonr_feature_selection(X_processed, y_processed, X_test_processed,np.array([1,-3,3]),np.array([0.95,0.025,0.025]))
    selected_features = data_relation.loc[indices_desc][abs(data_relation['correlation']) >= 0.1]
    X_selection = X_processed[:,selected_features.index]
    X_test_selection = X_test_processed[:,selected_features.index]
    X_selection_tree, X_test_selection_tree = select_features(X_selection, y_processed, X_test_selection,feature_num = 50, n_estimators= 100)
    X_selection_tree_cor, X_test_selection_tree_cor = pearsonr_feature_selection_correlation(X_selection_tree, X_test_selection_tree,0.96)
    return X_selection_tree_cor,X_test_selection_tree_cor

In [11]:
X_lgb,X_test_lgb = LGB_feature_selection(X_processed,X_test_processed,y_processed)
print(X_lgb.shape)
print(X_test_lgb.shape)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipyker

(1200, 98)
(776, 98)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match Da

In [12]:
X_gp_quad,X_test_gp_quad = GP_feature_selection(X_processed,X_test_processed,y_processed)
print(X_gp_quad.shape)
print(X_test_gp_quad.shape)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


(1200, 42)
(776, 42)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match Da

In [13]:
X_gp_rbf,X_test_gp_rbf = GP_rbf_feature_selection(X_processed,X_test_processed,y_processed)
print(X_gp_rbf.shape)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


(1200, 50)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match Da

In [14]:
x_tree,X_test_tree = Tree_feature_selection(X_processed,X_test_processed,y_processed)
print(x_tree.shape)
print(X_test_tree.shape)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


(1200, 49)
(776, 49)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match Da

## 4. Gaussian Process and lgb

In [15]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, DotProduct, WhiteKernel

def custom_r2(prediction, train_data):
    """Regular r2 cost function returned as a tuple to be used with lgb"""
    labels = train_data.get_label()
    return 'r2', r2_score(labels, prediction), True

def fit_model_and_pred(X_train, y_train, X_val, y_val, X_test):
#     kernel = RBF(0.5, (1e-4, 10))
#     gpr = GaussianProcessRegressor(kernel=kernel, random_state=0, n_restarts_optimizer=1, alpha=0.07, normalize_y=True)
#     gpr.fit(X_train, y_train)
#     score = gpr.score(X_val, y_val)   
#     y_pred = gpr.predict(X_test) 
    kernel = RationalQuadratic(length_scale=0.5, alpha=1.0, length_scale_bounds=(1e-4, 10))
    gpr = GaussianProcessRegressor(kernel=kernel, random_state=0, n_restarts_optimizer=1, alpha=1e-6, normalize_y=True)
    gpr.fit(X_train, y_train)
    score = gpr.score(X_val, y_val)
    y_val_pred = gpr.predict(X_val)   
    y_pred = gpr.predict(X_test)
    
    return score, y_pred

def fit_model_and_pred_gp2(X_train, y_train, X_val, y_val, X_test):
    # kernel = RBF(0.5, (1e-4, 10))
    kernel = Matern()
    gpr = GaussianProcessRegressor(kernel=kernel, random_state=0, n_restarts_optimizer=1, alpha=1e-4, normalize_y=True)
    gpr.fit(X_train, y_train)
    score = gpr.score(X_val, y_val)   
    y_pred = gpr.predict(X_test) 
    
    return score, y_pred

def fit_model_and_pred_tree(X_train, y_train, X_val, y_val, X_test, random_state=0):
    model = ExtraTreesRegressor(n_jobs=1, max_depth=None, n_estimators=190, random_state=random_state, min_samples_split=3, max_features=None)
    model.fit(X_train, y_train)
    
    y_val_pred = model.predict(X_val)
    score = r2_score(y_val, y_val_pred)
    
    y_pred = model.predict(X_test) 
    
    return score, y_pred

def fit_model_and_pred_lgb(X_train, y_train, X_val, y_val, X_test):
   
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'num_leaves': 1800,
        'learning_rate': 0.025,
        'max_depth': 11,
        'n_estimators': 1000,
        'min_child_weight': 1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'feature_fraction': 0.5,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': -1,
        'num_iterations':1000,
    }  
    
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=500,
                    feval=custom_r2,
                    valid_sets={lgb_train, lgb_eval},
                    early_stopping_rounds=100,
                    verbose_eval=False
                   )
    
    y_val_pred = gbm.predict(X_val)
    score = r2_score(y_val, y_val_pred)
    y_pred = gbm.predict(X_test) 
    return score, y_pred

def train_k_fold_2GP_LGB(X_lgb,X_test_lgb,X_gp,X_test_gp,X_tree,X_test_tree, y , weight1 = 0.5, weight2 = 0.75, fold_num=10):
# weight1 is the ratio of gp_rbf to lgb, weight2 is the combine of previous to gp.quadric
    kf = KFold(n_splits=fold_num, shuffle=False)
    kf.get_n_splits(X_lgb)
    test_score = 0.0
    train_score = 0.0
    y_test_pred = np.zeros((X_test_lgb.shape[0]))
#     print("shape of y_test_pred:", y_test_pred.shape)
    i = 0
    for train_index, test_index in kf.split(X_lgb):
        X_lgb_train, X_lgb_val = X_lgb[train_index], X_lgb[test_index]
        X_gp_train, X_gp_val = X_gp[train_index], X_gp[test_index]
        X_tree_train, X_tree_val = X_tree[train_index], X_tree[test_index]
        y_train, y_val = y[train_index], y[test_index]
        
        _, y_pred1 = fit_model_and_pred(X_gp_train, y_train, X_gp_val, y_val, X_gp_val)
        _, y_pred2 = fit_model_and_pred_lgb(X_lgb_train, y_train, X_lgb_val, y_val, X_lgb_val)
        _, y_pred3 = fit_model_and_pred_gp2(X_tree_train, y_train, X_tree_val, y_val, X_tree_val)
        _, y_pred4 = fit_model_and_pred_tree(X_tree_train, y_train, X_tree_val, y_val, X_tree_val)
        y_pred = weight2*(weight1*y_pred3+(1-weight1)*y_pred2) + (1-weight2)*y_pred1

        _, y_test_pred1 = fit_model_and_pred(X_gp_train, y_train, X_gp_val, y_val, X_test_gp)
        _, y_test_pred2 = fit_model_and_pred_lgb(X_lgb_train, y_train, X_lgb_val, y_val, X_test_lgb)
        _, y_test_pred3 = fit_model_and_pred_gp2(X_tree_train, y_train, X_tree_val, y_val, X_test_tree)
        _, y_test_pred4 = fit_model_and_pred_tree(X_tree_train, y_train, X_tree_val, y_val, X_test_tree)
        y_test_pred += weight2*(weight1*y_test_pred3+(1-weight1)*y_test_pred2) + (1-weight2)*y_test_pred1
        
        score = r2_score(y_val, y_pred)
        print(i+1, 'th . the obtained validation r2 score is : ',score)
        test_score += score
        i += 1

    return test_score/fold_num,y_test_pred/fold_num

def train_k_fold_predict(X, y, X_test, fold_num=10):
    kf = KFold(n_splits=fold_num)
    kf.get_n_splits(X)
    y_test_predict = np.zeros(X_test.shape[0])
    cnt = 0
    val_score = 0.0
    train_score = 0.0
    for train_index, test_index in kf.split(X):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]

        score, y_pred = fit_model_and_pred(X_train, y_train, X_val, y_val, X_test) 
        val_score += score
        y_test_predict += y_pred
        
        score, _ = fit_model_and_pred(X_train, y_train, X_train, y_train, X_test)
        train_score += score
    return train_score/fold_num, val_score/fold_num, y_test_predict/fold_num

In [31]:
score,prediction = train_k_fold_2GP_LGB(X_lgb,X_test_lgb,X_gp_quad,X_test_gp_quad,X_gp_rbf,X_test_gp_rbf, y_processed , weight1 = 0.578, weight2 = 0.68, fold_num=10)

/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8


/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
1 th . the obtained validation r2 score is :  0.7907270218264033


/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


...

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
9 th . the obtained validation r2 score is :  0.7632023798027


/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8


/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
10 th . the obtained validation r2 score is :  0.6226237572182121


In [32]:
score

0.6986158286897675

In [33]:
prediction

array([67.65583308, 69.12717282, 69.09128998, 70.58399406, 70.47578389,
       75.51179843, 62.13918408, 60.04690791, 81.13646267, 78.41051114,
       58.46246925, 81.88345073, 71.51511087, 81.69994332, 55.29428354,
       81.95810916, 70.08738681, 74.45204432, 71.47902602, 69.9356576 ,
       69.78893776, 66.09365384, 79.70548985, 75.36937   , 61.8986995 ,
       70.07189221, 55.28272352, 71.34309661, 65.53540596, 74.00201028,
       63.77711515, 69.70274477, 57.29171101, 70.54233039, 67.76422513,
       65.84495218, 74.94367368, 56.36728908, 71.50736518, 69.44520417,
       70.96438131, 74.85462846, 71.52226993, 79.76499194, 72.95379846,
       68.10992657, 69.21290362, 72.3219981 , 76.89870968, 70.47006028,
       76.6699029 , 65.94860981, 61.33230936, 82.50035943, 77.16444727,
       72.9732142 , 64.14192126, 71.46550767, 72.891654  , 60.97176867,
       65.25473237, 74.05970432, 64.79492025, 72.34482316, 72.92603114,
       58.27927285, 69.57578108, 78.81406279, 62.7327324 , 74.36

In [34]:
final_res = np.vstack((indices_test, prediction)).T
df_res = pd.DataFrame(final_res)
df_res.to_csv("our_result_rbf_quad_lgb2.csv", header = ["id", "y"], index=False)

In [35]:
# weight_trial = [0.68+0.01*i for i in range(0,15)]
# y_pred_trial = np.zeros((X_test_gp.shape[0],len(weight_trial)))
# score_trial = np.zeros(len(weight_trial))

# for i in range(0,len(weight_trial)):
#     score_trial[i],y_pred_trial[:,i] = train_k_fold_2GP_LGB(X_lgb,X_test_lgb,X_gp_quad,X_test_gp_quad,X_gp_rbf,X_test_gp_rbf, y_processed , weight1 = 0.578, weight2 = weight_trial[i], fold_num=10)

In [36]:
# feature_trial = [43 ]
# y_pred_trial = np.zeros((X_test_gp.shape[0],len(feature_trial)))
# score_trial = np.zeros(len(weight_trial))

# for i in range(0,len(weight_trial)):
#     X_gp_quad,X_test_gp_quad = GP_feature_selection(X_processed,X_test_processed,y_processed, tree_num = feature_trial[i])
#     score_trial[i],y_pred_trial[:,i] = train_k_fold_2GP_LGB(X_lgb,X_test_lgb,X_gp_quad,X_test_gp_quad,X_gp_rbf,X_test_gp_rbf, y_processed , weight1 = 0.578, weight2 = 0.69, fold_num=10)


In [37]:
# weight_trial = [0.57+0.002*i for i in range(0,6)]
# y_pred_trial = np.zeros((X_test_gp.shape[0],len(weight_trial)))
# score_trial = np.zeros(len(weight_trial))

# for i in range(0,len(weight_trial)):
#     score_trial[i],y_pred_trial[:,i] = train_k_fold_combine(X_lgb,X_test_lgb,X_gp,X_test_gp, y_processed , weight_trial[i], fold_num=10)

Experiment shows that the ratio of GP(with RBF kernals) to lgb is 0.578, the ratio of combination of above 2 to GP(quadratic) is 0.68. Experiment shows that good feature num for GP with quadratic is 43-45